In [25]:
# %%capture
# pip install econml shap "flaml[automl]" dill plotnine scikit-misc

In [26]:
import warnings
warnings.simplefilter('ignore')

In [27]:
import joblib
import dill as pickle

import pandas as pd
import numpy as np

from econml.dml import CausalForestDML, SparseLinearDML
from econml.dr import SparseLinearDRLearner, ForestDRLearner
from econml.metalearners import XLearner, TLearner, SLearner

from sklearn.base import BaseEstimator, clone
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LassoCV, LinearRegression, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.base import clone, BaseEstimator, clone
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold, StratifiedKFold, cross_val_predict, cross_val_score, train_test_split, GroupShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

import flaml
from flaml import AutoML

import matplotlib.pyplot as plt
import plotnine as p9
import shap
import math

import scipy
import scipy.special
from statsmodels.api import OLS
import os

# Load data

In [28]:
df = pd.read_parquet('../../output/pretraining/representations_combo_df_w_lags.parquet')

In [29]:
TIDE_VARS = [
    'very_low_7dr', 'low_7dr', 'in_range_7dr', 'g_7dr', 'using_pump',
    'in_range_7dr_7d_delta', 'large_tir_drop',
    'low_tir', 'lows', 'very_lows', 
    'pop_4T_1', 'pop_4T_2', 'pop_TIPS', 
]

In [30]:
# Drop rows with no flags
df['large_tir_drop'] = ((df['in_range_7dr_7d_delta'] < -0.15) & (df['time_worn_7dr'] > 0.5)).astype(int)
df['low_tir'] = ((df['in_range_7dr'] < 0.65) & (df['time_worn_7dr'] > 0.5)).astype(int)
df['lows'] = (df['low_7dr'] > 0.04).astype(int)
df['very_lows'] = (df['very_low_7dr'] > 0.01).astype(int)

print(df.shape)
df = df[(df.low_tir+df.lows+df.very_lows+df.large_tir_drop)>0]
print(df.shape)

(196816, 122)
(129345, 122)


# AutoML to fit nuisance models on TIDE variables

In [31]:
# We want to only use W variables when fitting and precting with nuisance models.
# In EconML, the X [features for CATE] and W [features for nuisances] are h-stacked. 
# We want to only us the W features (i.e. last W.shape[0] columns of XW)
# https://github.com/py-why/EconML/blob/main/econml/dml/dml.py#L44


###################################
# AutoML models
###################################

# FLAML models don't return "self" at end of fit. We create this wrapper.

class AutoMLWrap(BaseEstimator):

    def __init__(self, *, model, automl, features, filter_feats):
        self.model = model
        self.automl = automl
        self.features = features
        self.filter_feats = filter_feats

    def fit(self, X, y, **kwargs):
        self.model_ = clone(self.model)
        Xf = X[:,-len(self.features):] if self.filter_feats else X
        self.model_.fit(Xf, y, **kwargs)
        return self

    def predict(self, X):
        Xf = X[:,-len(self.features):] if self.filter_feats else X
        return self.model_.predict(Xf)

# Custom r2 loss for regression, for more trustworthy learning curves.
def reg_r2(
        X_val, y_val, estimator, labels,
        X_train, y_train, weight_val=None, weight_train=None,
        *args,):
    mse = np.mean((estimator.predict(X_val) - y_val)**2)
    r_2 = 1-mse/np.mean((y_val - y_val.mean())**2)
    return -1*r_2, {"val_loss": r_2}

def auto_reg(X, y, *, features,
             groups=None, n_splits=5, split_type='auto', time_budget=60, verbose=0, 
             estimator_list='auto', log_file_name='flaml_log.txt'):
    X = np.array(X)
    automl = AutoML(task='regression', time_budget=time_budget, early_stop=True,
                    eval_method='cv', n_splits=n_splits, split_type=split_type,
                    metric=reg_r2, verbose=verbose, estimator_list=estimator_list)
    if groups is None:
        automl.fit(X, y, log_file_name=log_file_name)
    else:
        automl.fit(X, y, groups=groups, log_file_name=log_file_name)
    best_est = automl.best_estimator
    return lambda x=False: AutoMLWrap(model=clone(automl.best_model_for_estimator(best_est)), automl=automl, features=features, filter_feats=x)




In [32]:
class AutoMLWrapCLF(BaseEstimator):

    def __init__(self, *, model, automl, prop_lb, features, filter_feats):
        self.model = model
        self.automl = automl
        self.prop_lb = prop_lb
        self.features = features
        self.filter_feats = filter_feats

    def fit(self, X, y, **kwargs):
        self.model_ = clone(self.model)
        Xf = X[:,-len(self.features):] if self.filter_feats else X
        self.model_.fit(Xf, y, **kwargs)
        return self

    def predict(self, X):
        Xf = X[:,-len(self.features):] if self.filter_feats else X
        preds = self.model_.predict_proba(Xf) 
        preds = np.clip(preds, self.prop_lb, 1-self.prop_lb)
        return preds
    
    def predict_proba(self, X):
        Xf = X[:,-len(self.features):] if self.filter_feats else X
        preds = self.model_.predict_proba(Xf) 
        preds = np.clip(preds, self.prop_lb, 1-self.prop_lb)
        return preds

# Custom r2 loss for classification, for more trustworthy learning curves.
def clf_r2(
        X_val, y_val, estimator, labels,
        X_train, y_train, weight_val=None, weight_train=None,
        *args,):
    mse = np.mean((estimator.predict_proba(X_val)[:, 1] - y_val)**2)
    r_2 = 1-mse/np.mean((y_val - y_val.mean())**2)
    return -1*r_2, {"val_loss": r_2}

def clf_mod_log_loss(
    X_val, y_val, estimator, labels,
    X_train, y_train, weight_val=None, weight_train=None,
    *args,):
    
    preds = estimator.predict_proba(X_val)[:,1]

    mod_log_loss = np.mean(-1* ( (.01 + y_val)*np.log(preds) + (1.01 - y_val)*np.log(1-preds)))

    return mod_log_loss, {"val_loss": mod_log_loss}

def auto_clf(
        X, y, *, features,
        groups=None, n_splits=5, split_type='auto', time_budget=60, verbose=0, estimator_list='auto', 
        log_file_name='flaml_log.txt', prop_lb=0.02):
    X = np.array(X)
    automl = AutoML(task='classification', time_budget=time_budget, early_stop=True,
                    eval_method='cv', n_splits=n_splits, split_type=split_type,
                    metric='log_loss', verbose=verbose, estimator_list=estimator_list,
                   )
    if groups is None:
        automl.fit(X, y, log_file_name=log_file_name)
    else:
        automl.fit(X, y, groups=groups, log_file_name=log_file_name)
    best_est = automl.best_estimator
    return lambda x=False: AutoMLWrapCLF(model=clone(automl.best_model_for_estimator(best_est)), automl=automl, prop_lb=prop_lb, features=features, filter_feats=x)

In [33]:
# Split data

df_train = df[df.data_split=='train'].copy()
df_val = df[df.data_split=='val'].copy()
df_test = df[df.data_split=='test'].copy()

In [ ]:
CONTROL_VARS = [
    'very_low_7dr', 'low_7dr', 'in_range_7dr', 'g_7dr', 'using_pump',
    'in_range_7dr_7d_delta', 'large_tir_drop',
    'low_tir', 'lows', 'very_lows', 
    'pop_4T_1', 'pop_4T_2', 'pop_TIPS'
]

LAGS_1W = ['lag_1_very_low_7dr','lag_1_low_7dr','lag_1_in_range_7dr','lag_1_g_7dr','lag_1_in_range_7dr_7d_delta','lag_1_large_tir_drop','lag_1_low_tir','lag_1_lows','lag_1_very_lows','lag_1_received_message']
LAGS_2W = ['lag_2_very_low_7dr','lag_2_low_7dr','lag_2_in_range_7dr','lag_2_g_7dr','lag_2_in_range_7dr_7d_delta','lag_2_large_tir_drop','lag_2_low_tir','lag_2_lows','lag_2_very_lows','lag_2_received_message']       
LAGS_3W = ['lag_3_very_low_7dr','lag_3_low_7dr','lag_3_in_range_7dr','lag_3_g_7dr','lag_3_in_range_7dr_7d_delta','lag_3_large_tir_drop','lag_3_low_tir','lag_3_lows','lag_3_very_lows','lag_3_received_message']       
LAGS_4W = ['lag_4_very_low_7dr','lag_4_low_7dr','lag_4_in_range_7dr','lag_4_g_7dr','lag_4_in_range_7dr_7d_delta','lag_4_large_tir_drop','lag_4_low_tir','lag_4_lows','lag_4_very_lows','lag_4_received_message']       

CONTROL_VARS = CONTROL_VARS + LAGS_1W + LAGS_2W + LAGS_3W #+ LAGS_4W
CONTROL_VARS

In [ ]:
# Fill NAs
fill_map = {x: 0 for x in CONTROL_VARS}
df_train = df_train.fillna(fill_map)
df_val = df_val.fillna(fill_map)
df_test = df_test.fillna(fill_map)

In [ ]:
# Fit nuisance models

TESTING = False
PROP_LB = 0.01

time_budget = 1 if TESTING else 300 
verbose = 1  # verbosity of auto-ml
n_splits = 10 # cross-fitting and cross-validation splits

X = df_train[CONTROL_VARS].astype(float).to_numpy()
Y = df_train['delta_in_range_fw_7d'].values
D = df_train['recommends_insulin_dose_change'].values
groups = df_train.mrn.values

# AutoML Models
model_y = auto_reg(
    X, Y, features=CONTROL_VARS, groups=groups, n_splits=n_splits, split_type='auto',
    verbose=verbose, time_budget=time_budget, estimator_list=['rf'])
model_t = auto_clf(
    X, D, features=CONTROL_VARS, groups=groups, n_splits=n_splits, split_type='auto',
    verbose=verbose, time_budget=time_budget, estimator_list=['rf'], prop_lb=PROP_LB)
model_reg_one = auto_reg(
    X[D==1], Y[D==1], features=CONTROL_VARS, groups=groups[D==1], n_splits=n_splits, split_type='auto',
    verbose=verbose, time_budget=time_budget, estimator_list=['rf'])

# # Load pickled stuff
# with open(f'../../output/analysis/reps_grid_search_5/model_y.pkl', 'rb') as f:
#     model_y = pickle.load(f)
    
# with open(f'../../output/analysis/reps_grid_search_5/model_t.pkl', 'rb') as f:
#     model_t = pickle.load(f)
    
# with open(f'../../output/analysis/reps_grid_search_5/model_reg_one.pkl', 'rb') as f:
#     model_reg_one = pickle.load(f)

# # Test
# print(model_y(True).fit(X, Y).predict(X))
# # print(model_reg_zero().fit(X, Y).predict(X))
# print(model_reg_one(True).fit(X, Y).predict(X))
# print(model_t(True).fit(X, D).predict(X))

# Shared Functions

In [16]:
# Calculate DR scores
def calculate_dr_outcomes(Xtrain, Dtrain, ytrain, groupstrain, Xval, Dval, yval, groupsval, model_t, model_reg_zero, model_reg_one, clipping):
    reg_zero = model_reg_zero().fit(Xtrain[list(Dtrain==0)], ytrain[list(Dtrain==0)])
    reg_one = model_reg_one().fit(Xtrain[list(Dtrain==1)], ytrain[list(Dtrain==1)])
    reg_zero_preds_t = reg_zero.predict(Xval)
    reg_one_preds_t = reg_one.predict(Xval)
    reg_preds_t = reg_zero_preds_t * (1 - Dval) + reg_one_preds_t * Dval
    prop_preds = model_t().fit(Xtrain, Dtrain).predict(Xval)[:,1]
    dr = reg_one_preds_t - reg_zero_preds_t
    reisz = (Dval - prop_preds) / np.clip(prop_preds * (1 - prop_preds), clipping, np.inf)
    dr += (yval - reg_preds_t) * reisz
    return dr


# Qini statistic
def calc_qini(cate_est, dr_scores, colsuffix):
    
    # Grid of values of % treated 5-100%
    ugrid = np.linspace(0, 95, 39)

    toc, toc_std, group_prob = np.zeros(len(qs)), np.zeros(len(qs)), np.zeros(len(qs))
    true_toc = np.zeros(len(qs))
    toc_psi = np.zeros((len(qs), dr_scores.shape[0]))
    n = len(dr_scores)
    ate = np.mean(dr_scores)
    for it in range(len(qs)):
        inds = (qs[it] <= cate_est) # group with larger CATE prediction than the q-th quantile
        group_prob = np.sum(inds) / n # fraction of population in this group
        toc[it] = group_prob * (np.mean(dr_scores[inds]) - ate) # tau(q) = q * E[Y(1) - Y(0) | tau(X) >= q[it]] - E[Y(1) - Y(0)]
        toc_psi[it, :] = (dr_scores - ate) * (inds - group_prob) - toc[it] # influence function for the tau(q)
        toc_std[it] = np.sqrt(np.mean(toc_psi[it]**2) / n) # standard error of tau(q)
    
    # Qini statistic
    qini_psi = np.sum(toc_psi[:-1] * np.diff(ugrid).reshape(-1, 1) / 100, 0)
    qini = np.sum(toc[:-1] * np.diff(ugrid) / 100)
    qini_stderr = np.sqrt(np.mean(qini_psi**2) / n)
    return(pd.DataFrame({
        f'QINI_{colsuffix}': [qini],
        f'QINI_se_{colsuffix}': [qini_stderr]
    }))


In [17]:
# Estimate CATE function
def est_cate_fn(estimator_name, prop_lb, model_y, model_t, Y, X, W, D, n_splits, groups):
    
    if estimator_name == 'CF':
        est = CausalForestDML(
            discrete_treatment=True,
            model_t=model_t(True),
            model_y=model_y(True),
            use_ray=False,
            cv=GroupKFold(n_splits=n_splits),
            random_state=123,
            # Same default params as grf Causal Forest
            n_estimators=4 if TESTING else 2000,
            min_balancedness_tol=0.45, # 0.05
            min_samples_split=1e-6,
            min_samples_leaf=20,
            max_features=min(math.ceil(math.sqrt(X.shape[1])) + 20, X.shape[1]),
            max_samples=0.5,
            subforest_size=2,
        )
        est.fit(Y, D, X=X, W=W, groups=groups)
    
    elif estimator_name == 'DRForest':
        est = ForestDRLearner(
            model_propensity=model_t(True),
            model_regression=model_y(True),
            random_state=123,
            cv=GroupKFold(n_splits=n_splits),
            min_propensity=prop_lb,
            n_estimators=4 if TESTING else 2000,
        )
        est.fit(Y, D, X=X, W=W, groups=groups)
        
    elif estimator_name == 'XLearner':
        est = XLearner(
            models = model_y(),
            cate_models = model_y(),
            propensity_model = model_t(True)
        )
        est.fit(Y, D, X=X)
    
    elif estimator_name == 'TLearner':
        est = TLearner(
            models = model_y(),
        )
        est.fit(Y, D, X=X)
        
    elif estimator_name == 'SLearner':
        est = SLearner(
            overall_model = model_y(),
        )
        est.fit(Y, D, X=X)
    
    else:
        raise NameError('Incorrect estimator_name in est_cate_fn')
    
    return(est)  


# Test All
# W = X
# Dm = df_train.custom_treatment.to_numpy()
# for x in ["CF","DRForest","XLearner","TLearner","SLearner"]:
#     print(x)
#     est_cate_fn(x, 0.01, model_y, model_t, Y, X, W, Dm, n_splits, groups)

In [18]:
# Calculate multinomial DR scores
def calculate_dr_outcomes_mn(Xtrain, Dtrain, ytrain, groupstrain, Xval, Dval, yval, groupsval, model_t, model_reg_zero, model_reg_one, clipping):
    
    n = Dval.shape[0]
    n_treatments = Dtrain.max()
    # print(f'n_treatments: {n_treatments}')
    
    # 0) Fit propensity model and make predictions
    reg_prop = model_t(True).fit(Xtrain, Dtrain)
    reg_prop_preds = reg_prop.predict(Xval)
    # print(reg_prop_preds.shape)
    
    # 1) Fit control outcome model
    reg_zero = model_reg_zero(True).fit(Xtrain[list(Dtrain==0)], ytrain[list(Dtrain==0)])
    
    # 2) Fit treatment outcome models for each treatment
    reg_t = {t: model_reg_one(True).fit(Xtrain[list(Dtrain==t)], ytrain[list(Dtrain==t)]) for t in range(1,n_treatments+1)}
    
    # 3) Create control and treatment predictions
    reg_zero_preds_t = reg_zero.predict(Xval)[:,np.newaxis]
    reg_t_preds = {t: reg_t[t].predict(Xval) for t in range(1,n_treatments+1)}
    reg_t_preds = np.column_stack(list(reg_t_preds.values()))
    
    # 4) Calculate differences in predicted outcomes for each treatment relative to control
    reg_tau_hat = reg_t_preds - np.tile(reg_zero_preds_t, (1,n_treatments))
    
    # 5) Calculate residuals
    reg_preds_t = np.hstack([reg_zero_preds_t, reg_t_preds])[np.arange(n),Dval]
    reg_res = yval - reg_preds_t
    
    # 6) Calculate IPW adjustment with residuals
    reg_prop_preds_t = reg_prop_preds[np.arange(n), Dval]
    ipws = np.where(Dval>0,1,-1) * (1/reg_prop_preds_t)
    ipw_res_adj = (reg_res*ipws)
    
    # 7) DR scores
    dr_scores = reg_tau_hat
    dr_scores[np.arange(n),Dval-1] += ipw_res_adj
    
    return(dr_scores)
    
# # TEST
# Dm = df_train.custom_treatment.to_numpy()
# Xval = df_val[TIDE_VARS].astype(float).to_numpy()
# Dval = df_val.custom_treatment.to_numpy()
# Yval = df_val['delta_in_range_fw_7d'].values
# groupsval = df_val.mrn.values

# test_dr = calculate_dr_outcomes_mn(X, Dm, Y, groups, Xval, Dval, Yval, groupsval, model_t, model_y, model_reg_one, clipping=0.01)
# print(test_dr)
# test_dr.mean(axis=0)

In [19]:
# Scorer
def drscore(cate_preds, dr_scores):
    
    # Only use max CATE preds
    max_cate_preds = cate_preds.max(axis=1)
    max_cate_pred_drs = dr_scores[np.arange(len(dr_scores)), np.argmax(cate_preds, axis=1)]
    
    overall_ate_val_dr = np.mean(max_cate_pred_drs)
    drscore_t = np.mean((max_cate_pred_drs - max_cate_preds)**2)
    drscore_b = np.mean((max_cate_pred_drs - overall_ate_val_dr)**2)
    return 1 - drscore_t / drscore_b


def get_cate_preds(est, X, n_treatments):
    cate_preds = []
    for t in range(1, n_treatments+1):
        cate_preds.append(est.effect(X,T1=t)[:,np.newaxis])
    cate_preds = np.hstack(cate_preds)
    return(cate_preds)

class Ensemble(BaseEstimator):

    def __init__(self, names, models, weights, n_treatments, intercept=0):
        self.names = names
        self.models = models
        self.weights = weights
        self.intercept = intercept
        self.n_treatments = n_treatments

    def predict(self, X):
        preds = [get_cate_preds(m, X, self.n_treatments) for m in self.models]
        w_preds = [w*p for w, p in zip(self.weights, preds)]
        wcate = np.stack(w_preds, axis=0).sum(axis=0)
        return self.intercept + wcate

In [20]:
# Ensemble CATE function
def est_ensemble_cate_fn(prop_lb, model_y, model_t, Y, X, W, D, n_splits, groups, Xval, dr_val_scores):
    
    # Estimate and score input models
    scorer = drscore
    score_name = 'DRscore'
    model_names = [
        'CF', 'DRForest', 'XLearner', 
        'TLearner', 'SLearner']
    models = [est_cate_fn(x, prop_lb, model_y, model_t, Y, X, W, D, n_splits, groups) for x in model_names]
    scores = [scorer(get_cate_preds(m, Xval, D.max()), dr_val_scores) for m in models]
    print([f'{name}: {score:.4f}' for name, score in zip(model_names, scores)])
    
    # Find the best ensemble based on val DR scores
    eta_grid = np.logspace(-5, 5, 10)
    ens = {}
    for eta in eta_grid:
        weights = scipy.special.softmax(eta * np.array(scores)).tolist()
        ensemble = Ensemble(model_names, models, weights, D.max())
        ens[eta] = (ensemble, scorer(ensemble.predict(Xval), dr_val_scores))

    score_best = -np.inf
    for eta in eta_grid:
        if ens[eta][1] >= score_best:
            score_best = ens[eta][1]
            eta_best = eta

    est = ens[eta_best][0]
    
    return(est)

# # Test
# W=X
# test_ensemble = est_ensemble_cate_fn(0.01, model_y, model_t, Y, X, W, Dm, n_splits, groups, Xval, test_dr)
# test_ensemble

In [21]:
# Calc AUTOC and also return TOC curve
def toc_df(cate_est, dr_scores, colsuffix):
    
    # Calc ATE for offset
    ate = np.mean(dr_scores)
    
    # Grid of values of % treated 5-95%
    ugrid = np.linspace(5, 95, 50)
    
    # Quantiles of CATE est corresponding to each treated %
    qs = np.percentile(cate_est, ugrid)

    # Initialize vectors
    toc, toc_std, group_prob = np.zeros(len(qs)), np.zeros(len(qs)), np.zeros(len(qs))
    toc_psi = np.zeros((len(qs), dr_scores.shape[0])) # influence function representation of the TOC at each quantile
    
    # Iterate over %'s
    n = len(dr_scores)
    for it in range(len(qs)):
        inds = (qs[it] <= cate_est) # subset with larger CATE prediction than the q-th quantile
        group_prob = np.sum(inds) / n # fraction of population in this group
        toc[it]= np.mean(dr_scores[inds]) - ate # Rel ATT at %
        # influence function for the tau(q); it is a standard influence function of a "covariance"
        toc_psi[it, :] = (dr_scores - ate) * (inds / group_prob - 1) - toc[it]
        toc_std[it] = np.sqrt(np.mean(toc_psi[it]**2) / n) # standard error of tau(q)
        
    toc_df = pd.DataFrame({
        'p_treated': 100 - ugrid,
        'toc': toc,
        'toc_std': toc_std,
        'toc_lb': toc - 1.96*toc_std,
        'toc_ub': toc + 1.96*toc_std,
        'att': toc + ate,
        'att_lb': toc + ate - 1.96*toc_std,
        'att_ub': toc + ate + 1.96*toc_std,
        'type': np.repeat([colsuffix], len(ugrid))
    })
        
    # Calculate AUTOC
    autoc_psi = np.sum(toc_psi[:-1] * np.diff(ugrid).reshape(-1, 1) / 100, 0)
    autoc = np.sum(toc[:-1] * np.diff(ugrid) / 100)
    autoc_stderr = np.sqrt(np.mean(autoc_psi**2) / n)
    autoc_df = pd.DataFrame({
        f'AUTOC_{colsuffix}': [autoc],
        f'AUTOC_se_{colsuffix}' : [autoc_stderr]
    })
    
    return({
        'autoc_df': autoc_df,
        'toc_df': toc_df
    })


# Adapt TOC function to handle multiple treatments (3 options + 'best cate pred')

def toc_df_mn(mn_cate_est, mn_dr_scores, colsuffix):
    
    # Get DR scores of max CATE preds
    # Combine with DR scores for other treatments
    max_test_dr = mn_dr_scores[np.arange(len(mn_dr_scores)), np.argmax(mn_cate_est, axis=1)]
    all_drs = np.hstack([mn_dr_scores, max_test_dr[:,np.newaxis]])
    # ATEs
    # print(all_drs.mean(axis=0))

    # Create CATE predictions matrix incl max
    max_cate_preds = mn_cate_est.max(axis=1)
    all_cate_preds = np.hstack([mn_cate_est, max_cate_preds[:,np.newaxis]])
    
    # Iterate over treatments and compute toc_df separately for each one
    toc_dfs = []
    autoc_dfs = []
    for t in range(all_cate_preds.shape[1]):
        res = toc_df(all_cate_preds[:,t], all_drs[:,t], f'{colsuffix}_t{t+1}')
        autoc_dfs.append(res['autoc_df'])
        toc_dfs.append(res['toc_df'])
        
    toc_dfs = pd.concat(toc_dfs, axis=0)
    autoc_dfs = pd.concat(autoc_dfs, axis=1)
    return({
        'autoc_df': autoc_dfs,
        'toc_df': toc_dfs
    })

# Bootstrap

def toc_df_mn_bs(mn_cate_est, mn_dr_scores, groups, n_bootstrap_samples, colsuffix):
    
    unique_groups = np.unique(groups)
    bootstrap_toc_dfs = []
    for i in range(n_bootstrap_samples):
        sampled_groups = np.random.choice(unique_groups, len(unique_groups), replace=True)
        sampled_mn_cate_est = np.vstack([
            mn_cate_est[groups == g] for g in sampled_groups])
        sampled_mn_dr_scores = np.vstack([
            mn_dr_scores[groups == g] for g in sampled_groups])
        
        res_df = toc_df_mn(sampled_mn_cate_est, sampled_mn_dr_scores, colsuffix)
        res_df['toc_df']['bs_id'] = i+1
        res_df['autoc_df']['bs_id'] = i+1
        bootstrap_toc_dfs.append(res_df)
        
    toc_df = pd.concat([x['toc_df'] for x in bootstrap_toc_dfs])
    autoc_df = pd.concat([x['autoc_df'] for x in bootstrap_toc_dfs])
    
    # Aggregate across bootstrap samples to get STD and LB UB
    
    toc_df = toc_df.groupby(['p_treated','type'])[['toc','att']].agg(['mean','std']).reset_index()
    toc_df.columns = ['p_treated','type','toc','toc_std','att','att_std']
    toc_df['toc_lb'] = toc_df.toc - 1.96*toc_df.toc_std
    toc_df['toc_ub'] = toc_df.toc + 1.96*toc_df.toc_std
    toc_df['att_lb'] = toc_df.att - 1.96*toc_df.att_std
    toc_df['att_ub'] = toc_df.att + 1.96*toc_df.att_std
    toc_df = toc_df[['p_treated','toc','toc_std','toc_lb','toc_ub','att','att_lb','att_ub','type']]
    
    n_treatments = mn_cate_est.shape[1]+1
    autoc_cols_to_agg = [f'AUTOC_{colsuffix}_t{t+1}' for t in range(n_treatments)]
    autoc_means = autoc_df[autoc_cols_to_agg].agg('mean')
    autoc_se = autoc_df[autoc_cols_to_agg].agg('std')
    autoc_df = pd.DataFrame([autoc_means])
    for t in range(n_treatments):
        autoc_df[f'AUTOC_se_{colsuffix}_t{t+1}'] = autoc_se[f'AUTOC_{colsuffix}_t{t+1}']
    
    return({
        'toc_df': toc_df,
        'autoc_df': autoc_df
    })

# # TEST
# (p9.ggplot(toc_df_mn(est_cates, test_dr, 'val')['toc_df'],
#            p9.aes(x='p_treated', y='att', ymin='att_lb', ymax='att_ub', color='type')) + 
#      p9.geom_line() + p9.geom_pointrange() + p9.theme_bw() + p9.geom_hline(yintercept=0, linetype='dashed'))

# Variable Logic


In [47]:
STATE_REPS = {
    'TIDE (+3w lags in control covars)': TIDE_VARS,
    'TIDE w Lags': TIDE_VARS + ['lag_1_very_low_7dr','lag_1_low_7dr','lag_1_in_range_7dr','lag_1_g_7dr','lag_1_in_range_7dr_7d_delta','lag_1_large_tir_drop','lag_1_low_tir','lag_1_lows','lag_1_very_lows','lag_1_received_message','lag_2_very_low_7dr','lag_2_low_7dr','lag_2_in_range_7dr','lag_2_g_7dr','lag_2_in_range_7dr_7d_delta','lag_2_large_tir_drop','lag_2_low_tir','lag_2_lows','lag_2_very_lows','lag_2_received_message','lag_3_very_low_7dr','lag_3_low_7dr','lag_3_in_range_7dr','lag_3_g_7dr','lag_3_in_range_7dr_7d_delta','lag_3_large_tir_drop','lag_3_low_tir','lag_3_lows','lag_3_very_lows','lag_3_received_message','lag_4_very_low_7dr','lag_4_low_7dr','lag_4_in_range_7dr','lag_4_g_7dr','lag_4_in_range_7dr_7d_delta','lag_4_large_tir_drop','lag_4_low_tir','lag_4_lows', 'lag_4_very_lows', 'lag_4_received_message'],
    'TIDE 1w Lags': TIDE_VARS + 
        ['lag_1_very_low_7dr','lag_1_low_7dr','lag_1_in_range_7dr','lag_1_g_7dr','lag_1_in_range_7dr_7d_delta','lag_1_large_tir_drop','lag_1_low_tir','lag_1_lows','lag_1_very_lows','lag_1_received_message'],
    'TIDE 2w Lags': TIDE_VARS + \
        ['lag_1_very_low_7dr','lag_1_low_7dr','lag_1_in_range_7dr','lag_1_g_7dr','lag_1_in_range_7dr_7d_delta','lag_1_large_tir_drop','lag_1_low_tir','lag_1_lows','lag_1_very_lows','lag_1_received_message'] + \
        ['lag_2_very_low_7dr','lag_2_low_7dr','lag_2_in_range_7dr','lag_2_g_7dr','lag_2_in_range_7dr_7d_delta','lag_2_large_tir_drop','lag_2_low_tir','lag_2_lows','lag_2_very_lows','lag_2_received_message'],
    'Expert (full)': ['g_7dr', 'very_low_7dr', 'low_7dr', 'in_range_7dr', 'high_7dr', 'very_high_7dr', 'gri_7dr', 'g_14dr', 'very_low_14dr', 'low_14dr', 'in_range_14dr', 'high_14dr', 'very_high_14dr', 'gri_14dr', 'night_very_low_7dr', 'night_low_7dr', 'night_high_7dr', 'night_very_high_7dr', 'day_very_low_7dr', 'day_low_7dr', 'day_high_7dr', 'day_very_high_7dr', 'time_worn_7dr', 'night_worn_7dr', 'day_worn_7dr', 'sexF', 'public_insurance', 'english_primary_language', 'pop_pilot', 'pop_4T_1', 'pop_4T_2', 'pop_TIPS', 'age', 'months_since_onset', 'using_pump', 'using_aid', 'large_tir_drop', 'low_tir', 'lows', 'very_lows'],
    'Expert (subset)': ['large_tir_drop', 'in_range_14dr', 'in_range_7dr', 'low_7dr', 'using_pump', 'time_worn_7dr', 'day_worn_7dr', 'day_low_7dr', 'g_7dr', 'months_since_onset'],
    'Raw (UMAP) (+3w lags in control covars)': ['umap0', 'umap1', 'umap2', 'umap3'],
    'Raw (TS2Vec) (+3w lags in control covars)': ['ts2vec_0', 'ts2vec_1', 'ts2vec_2', 'ts2vec_3', 'ts2vec_4', 'ts2vec_5',
       'ts2vec_6', 'ts2vec_7']
}

ACTION_REPS = {
    'Expert v3': 'custom_treatment_v3',
    'Binary (any)': 'received_message',
    'Clustered Embeddings (2)': 'km_treatment_2',
    'Clustered Embeddings (3)': 'km_treatment_3',
    'Clustered Embeddings (4)': 'km_treatment_4',
}

In [44]:
def driver(df, outcome, state_rep, action_rep, 
           cate_estimator, p_wins, prop_lb):
    
    ####################
    ### PREPARE DATA ###
    ####################
    
    # Define reward (outcome) and winsorize
    for d in [df_train, df_val, df_test]:
        d['reward'] = d[outcome]
    lb, ub = df_train.reward.quantile([p_wins, 1-p_wins])
    for d in [df_train, df_val, df_test]:
        d['reward'] = d.reward.clip(lb, ub)

    X = df_train[STATE_REPS[state_rep]].astype(float).fillna(0.0).to_numpy()
    W = df_train[CONTROL_VARS].astype(float).to_numpy()
    Y = df_train['reward'].values
    D = df_train[ACTION_REPS[action_rep]].astype(int).values
    groups = df_train.mrn.values

    Xval = df_val[STATE_REPS[state_rep]].astype(float).fillna(0.0).to_numpy()
    Wval = df_val[CONTROL_VARS].astype(float).to_numpy()
    Yval = df_val['reward'].values
    Dval = df_val[ACTION_REPS[action_rep]].astype(int).values
    groupsval = df_val.mrn.values

    Xtest = df_test[STATE_REPS[state_rep]].astype(float).fillna(0.0).to_numpy()
    Wtest = df_test[CONTROL_VARS].astype(float).to_numpy()
    Ytest = df_test['reward'].values
    Dtest = df_test[ACTION_REPS[action_rep]].astype(int).values
    groupstest = df_test.mrn.values
    
    # print(groups.shape)
    # print(np.unique(groups).shape)
    # print(groupsval.shape)
    # print(np.unique(groupsval).shape)
    # print(groupstest.shape)
    # print(np.unique(groupstest).shape)
    
    #######################################
    ### Fit and Evaluate CATE Estimator ###
    #######################################
    
    # Calculate DR scores
    dr_val = calculate_dr_outcomes_mn(W, D, Y, groups, Wval, Dval, Yval, groupsval, model_t, model_y, model_reg_one, clipping=prop_lb)
    dr_test = calculate_dr_outcomes_mn(W, D, Y, groups, Wtest, Dtest, Ytest, groupstest, model_t, model_y, model_reg_one, clipping=prop_lb)

    # Train estimator
    if cate_estimator != 'Ensemble':
        est = est_cate_fn(cate_estimator, prop_lb, model_y, model_t, Y, X, W, D, n_splits, groups)
    else:
        est = est_ensemble_cate_fn(prop_lb, model_y, model_t, Y, X, W, D, n_splits, groups, Xval, dr_val)
    
    # Make CATE predictions
    if cate_estimator != 'Ensemble':
        cate_preds_val = []
        for t in range(1, D.max()+1):
            cate_preds_val.append(est.effect(Xval,T1=t)[:,np.newaxis])
        cate_preds_val = np.hstack(cate_preds_val)
        
        cate_preds_test = []
        for t in range(1, D.max()+1):
            cate_preds_test.append(est.effect(Xtest,T1=t)[:,np.newaxis])
        cate_preds_test = np.hstack(cate_preds_test)
        
    else:
        cate_preds_val = est.predict(Xval)
        cate_preds_test = est.predict(Xtest)

    # AUTOC and TOC
    toc_val = toc_df_mn_bs(cate_preds_val, dr_val, groupsval, 500, 'val')
    toc_test = toc_df_mn_bs(cate_preds_test, dr_test, groupstest, 500, 'test')
    
    ################################
    ### Create output dataframes ###
    ################################
    
    params_df = pd.DataFrame(
        [[outcome, state_rep, action_rep, cate_estimator, p_wins, prop_lb]],
        columns=['outcome', 'state_rep', 'action_rep', 'cate_estimator', 'p_wins', 'prop_lb'])
    
    # Combine dataframes
    summary_df = pd.concat(
        [params_df, toc_val['autoc_df'], toc_test['autoc_df'], 
         # lproj_df
        ],
        axis=1)
    
    toc_df_val = toc_val['toc_df']
    toc_df_test = toc_test['toc_df']
    toc_df_val['dataset'] = 'val'
    toc_df_test['dataset'] = 'test'
    toc_df = pd.concat([toc_df_val, toc_df_test], ignore_index=True)
    toc_df = toc_df.assign(
        outcome = np.repeat(outcome, len(toc_df)),
        state_rep = np.repeat(state_rep, len(toc_df)),
        action_rep = np.repeat(action_rep, len(toc_df)),
        cate_estimator = np.repeat(cate_estimator, len(toc_df)),
        p_wins = np.repeat(p_wins, len(toc_df)),
        prop_lb = np.repeat(prop_lb, len(toc_df)),
    )

    return({
        'summary_df': summary_df,
        'toc_df': toc_df
    })


# # TEST
# TESTING = False
# t = driver(
#     df,
#     outcome = 'delta_in_range_fw_7d',
#     state_rep = 'TIDE',
#     action_rep = 'Expert v2',
#     cate_estimator = 'Ensemble',
#     p_wins = 0.1,
#     prop_lb = 0.02
# )
# pd.set_option('display.max_columns', 50)
# print(t['summary_df'])
# t['toc_df']

In [ ]:
# Define Grid

from itertools import product

outcomes = [
    'delta_in_range_fw_7d', 
]
state_reps = list(STATE_REPS.keys())
action_reps = list(ACTION_REPS.keys())
cate_estimators = [
    'Ensemble',
    'TLearner', 
    'CF', 
    'DRForest', 
    'XLearner', 
    'SLearner',
]
p_wins = [0.05]
prop_lb = [0.01]

grid = list(product(
    outcomes, state_reps, action_reps, cate_estimators, p_wins, prop_lb
))

print(len(grid))

In [ ]:
# Run whole grid
TESTING = False
N_SUCCESSES = 0
PREFIX=''
FOLDER='eval_results'

output_dir = FOLDER
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  

# for i in range(len(grid)):
for i in range(len(grid)):
    print(i)
    print(grid[i])
    res = driver(df, *grid[i])
    try:
        res = driver(df, *grid[i])
        res['summary_df'].to_csv(f'{output_dir}/{PREFIX}_{i}_summary.csv', index=False)
        res['toc_df'].to_csv(f'{output_dir}/{PREFIX}_{i}_toc.csv', index=False)
        N_SUCCESSES += 1
        print(f'{N_SUCCESSES} of {len(grid)} runs succeeded so far')
    except:
        print("ERROR")

print('DONE')

0
('delta_in_range_fw_7d', 'TIDE', 'Expert', 'CF', 0.1, 0.01)
1 of 180 runs succeeded so far
1
('delta_in_range_fw_7d', 'TIDE', 'Expert', 'DRForest', 0.1, 0.01)
2 of 180 runs succeeded so far
2
('delta_in_range_fw_7d', 'TIDE', 'Expert', 'XLearner', 0.1, 0.01)
3 of 180 runs succeeded so far
3
('delta_in_range_fw_7d', 'TIDE', 'Expert', 'TLearner', 0.1, 0.01)
4 of 180 runs succeeded so far
4
('delta_in_range_fw_7d', 'TIDE', 'Expert', 'SLearner', 0.1, 0.01)
5 of 180 runs succeeded so far
5
('delta_in_range_fw_7d', 'TIDE', 'Binary (any)', 'CF', 0.1, 0.01)
6 of 180 runs succeeded so far
6
('delta_in_range_fw_7d', 'TIDE', 'Binary (any)', 'DRForest', 0.1, 0.01)
7 of 180 runs succeeded so far
7
('delta_in_range_fw_7d', 'TIDE', 'Binary (any)', 'XLearner', 0.1, 0.01)
8 of 180 runs succeeded so far
8
('delta_in_range_fw_7d', 'TIDE', 'Binary (any)', 'TLearner', 0.1, 0.01)
9 of 180 runs succeeded so far
9
('delta_in_range_fw_7d', 'TIDE', 'Binary (any)', 'SLearner', 0.1, 0.01)
10 of 180 runs succeede

OpenBLAS warning: precompiled NUM_THREADS exceeded, adding auxiliary array for thread metadata.


26 of 180 runs succeeded so far
26
('delta_in_range_fw_7d', 'TIDE', 'Clustered Embeddings (4)', 'DRForest', 0.1, 0.01)
27 of 180 runs succeeded so far
27
('delta_in_range_fw_7d', 'TIDE', 'Clustered Embeddings (4)', 'XLearner', 0.1, 0.01)
28 of 180 runs succeeded so far
28
('delta_in_range_fw_7d', 'TIDE', 'Clustered Embeddings (4)', 'TLearner', 0.1, 0.01)
29 of 180 runs succeeded so far
29
('delta_in_range_fw_7d', 'TIDE', 'Clustered Embeddings (4)', 'SLearner', 0.1, 0.01)
30 of 180 runs succeeded so far
30
('delta_in_range_fw_7d', 'Expert (full)', 'Expert', 'CF', 0.1, 0.01)
31 of 180 runs succeeded so far
31
('delta_in_range_fw_7d', 'Expert (full)', 'Expert', 'DRForest', 0.1, 0.01)
32 of 180 runs succeeded so far
32
('delta_in_range_fw_7d', 'Expert (full)', 'Expert', 'XLearner', 0.1, 0.01)
33 of 180 runs succeeded so far
33
('delta_in_range_fw_7d', 'Expert (full)', 'Expert', 'TLearner', 0.1, 0.01)
34 of 180 runs succeeded so far
34
('delta_in_range_fw_7d', 'Expert (full)', 'Expert', 'SL